This is the script that should be run after constructing the main dataset to further clean the dataset of internal defects.

In [1]:
from replica_processor import *

Welcome to JupyROOT 6.24/06


### Script to fix duplicate values

In [2]:
dfs = pd.read_csv('dfs.csv', dtype={'x':'int8','y':'int8','ftag_id':'int8','occ':'float32','hist_type':'int8','hist_id':'int16'})

Determine and store a list of ftag_ids that contain duplicate data points to clean

In [4]:
# Initialize the list that will contain the reduced list of offended ftags that contain duplicate datapoints
tags_to_clean = []

# Loop through ftags in the dataset
for ftag in dfs['ftag_id'].unique():
    
    # Get a handle for that ftag subset
    tmp = dfs[dfs['ftag_id']==ftag]
    
    # Loop through the histogram's datapoints in that ftag
    for hist in tmp['hist_id'].unique():
        
        # Get a handle for that histogram's datapoints in that ftag
        tmp2 = tmp[tmp['hist_id']==hist]
        
        # Determine how many sets of extra datapoints there are per histogram (1.0 implies double the datapoints, 2 implies triple, etc.)
        val = tmp2.shape[0]/6435-1
        
        # If there are duplicate datapoints
        if val>0:
            # Display the information relevant to those duplicate datapoints such as the ftag they come from, histid they come from, # datapoints, and #sets of extra datapoints
            print(ftag,hist,tmp2.shape[0],tmp2.shape[0]/6435-1)
            
            # If this ftag has not been found to previously have duplicate data points, add that ftag to the list of ftags to clean for the next step
            if ftag not in tags_to_clean:
                tags_to_clean.append(ftag)

# Display the ftags that have been found to need duplicate data point removal
display(tags_to_clean);

1 126 25740 3.0
1 127 25740 3.0
1 128 25740 3.0
1 129 25740 3.0
1 130 25740 3.0
1 131 25740 3.0
1 132 25740 3.0
1 133 25740 3.0
1 134 25740 3.0
1 135 25740 3.0
1 136 25740 3.0
1 137 25740 3.0
1 138 25740 3.0
1 139 25740 3.0
1 140 25740 3.0
1 141 25740 3.0
1 142 25740 3.0
1 143 25740 3.0
1 144 25740 3.0
1 145 25740 3.0
1 146 25740 3.0
1 147 25740 3.0
1 148 25740 3.0
1 149 25740 3.0
1 150 25740 3.0
1 151 25740 3.0
1 152 25740 3.0
1 153 25740 3.0
1 154 25740 3.0
1 155 25740 3.0
1 156 25740 3.0
1 157 25740 3.0
1 158 25740 3.0
1 159 25740 3.0
1 160 25740 3.0
1 161 25740 3.0
1 162 25740 3.0
1 163 25740 3.0
1 164 25740 3.0
1 165 25740 3.0
1 166 25740 3.0
1 167 25740 3.0
1 168 25740 3.0
1 169 25740 3.0
1 170 25740 3.0
1 171 25740 3.0
1 172 25740 3.0
1 173 25740 3.0
1 174 25740 3.0
1 175 25740 3.0
1 176 25740 3.0
1 177 25740 3.0
1 178 25740 3.0
1 179 25740 3.0
1 180 25740 3.0
1 181 25740 3.0
1 182 25740 3.0
1 183 25740 3.0
1 184 25740 3.0
1 185 25740 3.0
1 186 25740 3.0
1 187 25740 3.0
1 188 25

[1, 45, 46, 52, 55, 56, 57, 60, 61, 62, 63, 64, 65, 69, 70, 82, 84]

How many histograms are there in each ftag_id that contains duplicates - in case we are interested

In [5]:
for ftag in dfs['ftag_id'].unique():
    tmp = dfs[dfs['ftag_id']==ftag]
    if ftag in tags_to_clean:
        print(ftag,tmp['hist_id'].max()+1)

0 36
1 198
2 18
3 18
4 18
5 18
6 18
7 18
8 18
9 36
10 18
11 18
12 18
13 18
14 18
17 36
18 18
19 18
20 18
23 36
24 18
25 36
26 36
29 36
30 18
31 36
32 18
33 18
35 18
36 18
37 18
38 18
39 18
40 18
41 18
42 18
43 18
45 108
46 72
47 54
48 126
49 36
50 36
51 216
52 198
53 18
54 54
55 108
56 108
57 288
58 198
59 216
60 162
61 54
62 360
63 180
64 18
65 108
66 90
67 18
68 36
69 54
70 126
71 18
72 72
73 18
74 234
75 72
76 18
77 162
78 216
79 18
80 324
81 18
82 414
83 90
84 234
85 18
86 54
87 36
88 126
89 180
90 54


Drop the duplicate data points from the offending ftag_ids

In [6]:
# Delete clean_dfs if it already exists
try:
    del clean_dfs
except:
    pass


# loop through ftags
for idf,ftag in enumerate(range(dfs['ftag_id'].max()+1)):
    
    # Display the ftag thats processing
    print('ftag:',ftag)
    
    # Skip ftags we aren't cleaning
    if ftag not in tags_to_clean:
        continue
        
    # Display the progress to completing this ftag to process
    progress_bar(idf,range(dfs['ftag_id'].max()+1))
    
    # Get a handle for the ftag we are cleaning
    tmp = dfs[dfs['ftag_id']==ftag]
    
    # Loop through hists
    for idh,hist_id in enumerate(range(tmp['hist_id'].max()+1)):
        
        # Display which histogram is processing
        print(idh,'of',len(range(tmp['hist_id'].max()+1)))
        
        # Get a handle for the histogram we are cleaning
        tmp2 = tmp[tmp['hist_id']==hist_id]
        
        # Loop through the x coords, this will give us access to the subset of ftag|hist|x vals where there are
        # 130 datapoints when there should be 65, drop duplicates effectively removes the problem in each subset
        for xcoord in range(99):
            # Get a handle for the strip to clean in the histogram
            tmp3 = tmp2[tmp2['x']==xcoord]
            
            # Drop the duplicates in that strip for each duplicate y value
            tmp3 = tmp3.drop_duplicates(subset='y')
            
            # Move the cleaned values to their own dataframe
            try: # If the dataframe exists, add the new values to it
                clean_dfs = pd.concat([clean_dfs,tmp3])
            except: # If not, initialize the dataframe with the first values
                clean_dfs = tmp3

# This process takes long enough where we want to dump the data to a csv file right after in case we cant be
# present to immediately save it
clean_dfs.to_csv('clean_dfs.csv', index=False)

Processing file 85 of 91 files... 93.41% Complete
0 of 234
1 of 234
2 of 234
3 of 234
4 of 234
5 of 234
6 of 234
7 of 234
8 of 234
9 of 234
10 of 234
11 of 234
12 of 234
13 of 234
14 of 234
15 of 234
16 of 234
17 of 234
18 of 234
19 of 234
20 of 234
21 of 234
22 of 234
23 of 234
24 of 234
25 of 234
26 of 234
27 of 234
28 of 234
29 of 234
30 of 234
31 of 234
32 of 234
33 of 234
34 of 234
35 of 234
36 of 234
37 of 234
38 of 234
39 of 234
40 of 234
41 of 234
42 of 234
43 of 234
44 of 234
45 of 234
46 of 234
47 of 234
48 of 234
49 of 234
50 of 234
51 of 234
52 of 234
53 of 234
54 of 234
55 of 234
56 of 234
57 of 234
58 of 234
59 of 234
60 of 234
61 of 234
62 of 234
63 of 234
64 of 234
65 of 234
66 of 234
67 of 234
68 of 234
69 of 234
70 of 234
71 of 234
72 of 234
73 of 234
74 of 234
75 of 234
76 of 234
77 of 234
78 of 234
79 of 234
80 of 234
81 of 234
82 of 234
83 of 234
84 of 234
85 of 234
86 of 234
87 of 234
88 of 234
89 of 234
90 of 234
91 of 234
92 of 234
93 of 234
94 of 234
95 of 234


The clean_dfs dataframe only contains the histograms in the ftags listed in tags_to_clean. We now collect the hists from the non cleaned ftags and combine them together for the final cleaned dataset.

In [10]:
for idf,ftag in enumerate(dfs['ftag_id'].unique()):
    
    progress_bar(idf,dfs['ftag_id'].unique())
    print('ftag:',ftag)
    
    if ftag in tags_to_clean:
        continue
        
    tmp = dfs[dfs['ftag_id']==ftag]
    
    try:
        non_cleaned_df = pd.concat([non_cleaned_df,tmp])
    except:
        non_cleaned_df = tmp

Processing file 83 of 83 files... 100.0% Complete
ftag: 90


combine the cleaned and not cleaned ftag datapoints together to create the duplicate cleaned dataset - main_dfs

In [14]:
main_dfs = pd.concat([non_cleaned_df,clean_dfs])

In [15]:
main_dfs.shape #should be near 44 million (#hists*65*99)

(41235480, 7)

In [16]:
main_dfs.to_csv('main_dfs.csv',index=False)

In [17]:
main_dfs

,paths,x,y,occ,ftag_id,hist_type,hist_id
0,run_363664/CaloMonitoring/ClusterMon/CaloCalTo...,0,0,0.0,0,0,0
1,run_363664/CaloMonitoring/ClusterMon/CaloCalTo...,0,1,0.0,0,0,0
2,run_363664/CaloMonitoring/ClusterMon/CaloCalTo...,0,2,0.0,0,0,0
3,run_363664/CaloMonitoring/ClusterMon/CaloCalTo...,0,3,0.0,0,0,0
4,run_363664/CaloMonitoring/ClusterMon/CaloCalTo...,0,4,0.0,0,0,0
...,...,...,...,...,...,...,...
60694915,run_360402/CaloMonitoring/ClusterMon/LArCluste...,98,60,0.0,84,17,233
60694916,run_360402/CaloMonitoring/ClusterMon/LArCluste...,98,61,0.0,84,17,233
60694917,run_360402/CaloMonitoring/ClusterMon/LArCluste...,98,62,0.0,84,17,233
60694918,run_360402/CaloMonitoring/ClusterMon/LArCluste...,98,63,0.0,84,17,233


We can now move to the script that will generate the anomalies in the noisy dataset or go straight to the normalization script to generate the non noisy dataset.